In [1]:
import osmnx as ox
import numpy as np
import pandas as pd
import geopandas as gp
import folium
from shapely.geometry import Point, Polygon
import altair as alt

<p style = "color: blue; font-size :40px;text-align:center;font-family:Times New Roman"> Tarea 4

# 1. [30 puntos] Utilizando la biblioteca `altair` y la tabla de datos `movies` de la biblioteca `vega_datasets` realice lo siguiente:

In [9]:
import altair as alt
from vega_datasets import data

movies =data.movies()
movies = movies[["Release_Date","Title","Major_Genre","Rotten_Tomatoes_Rating","Production_Budget"]]
movies.Title = movies.Title.astype(str)
movies.Release_Date = pd.to_datetime(movies.Release_Date)
movies = movies.dropna()
movies

,Release_Date,Title,Major_Genre,Rotten_Tomatoes_Rating,Production_Budget
3,1998-09-11,Let's Talk About Sex,Comedy,13.0,300000.0
4,1998-10-09,Slam,Drama,62.0,1000000.0
11,1968-12-11,Oliver!,Musical,84.0,10000000.0
21,1972-11-09,1776,Drama,57.0,4000000.0
22,1979-12-14,1941,Comedy,33.0,32000000.0
...,...,...,...,...,...
3196,2008-10-31,Zack and Miri Make a Porno,Comedy,65.0,24000000.0
3197,2007-03-02,Zodiac,Thriller/Suspense,89.0,85000000.0
3198,2006-08-11,Zoom,Adventure,3.0,35000000.0
3199,2005-10-28,The Legend of Zorro,Adventure,26.0,80000000.0


## a) Utilizando la biblioteca `altair` diseñe un gráfico de barras que permita visualizar el promedio del presupuesto invertido por genero cinematográfico e interpreta los resultados.

In [12]:
alt.Chart(movies,
                 title = "Relación año y presupuesto de producción."
                 ).mark_bar(point = { "size": 80},
                             strokeWidth = 5,
                             color = "#c67ace").encode(
    x = alt.X('Major_Genre', title = "Género"),
    y = alt.Y('mean(Production_Budget):Q', 
              title = "presupuesto",
              scale = alt.Scale(zero = False, type = "log")),
    color = alt.Color('Major_Genre', 
                      legend = alt.Legend(title = "Género principal")),
    tooltip = ['year(Release_Date):T',
               'min(Production_Budget):Q',
               'mean(Production_Budget):Q',
               'median(Production_Budget):Q',
               'max(Production_Budget):Q']).properties(width=650, height=400
    ).interactive()

alt.Chart(...)

## b) Utilizando un selector de slider modifique el gráfico para que permita seleccionar el año de los datos que se desean visualizar.

In [19]:
year_slider = alt.binding_range(min=1969, max=2010, step=1)
slider_selection = alt.selection_single(bind = year_slider,
                                        fields = ['Release_Year'],
                                        name = "Select")

plot = alt.Chart(movies,
                 title = "Relación año y presupuesto de producción."
                 ).mark_bar(point = { "size": 80},
                             strokeWidth = 5,
                             color = "#c67ace").encode(
    x = alt.X('Major_Genre', title = "Género"),
    y = alt.Y('mean(Production_Budget):Q', 
              title = "presupuesto",
              scale = alt.Scale(zero = False, type = "log")),
    color = alt.Color('Major_Genre', 
                      legend = alt.Legend(title = "Género principal")),
    tooltip = ['year(Release_Date):T',
               'min(Production_Budget):Q',
               'mean(Production_Budget):Q',
               'median(Production_Budget):Q',
               'max(Production_Budget):Q']).properties(width=650, height=400
    ).add_selection(
    slider_selection).transform_filter(
    slider_selection).transform_calculate(Release_Year = "year(datum.Release_Date)"
)
                      
plot = plot.configure(
  title = alt.TitleConfig(
    fontSize = 22,
    align = "center",
    color = "#364547"),
  axis = alt.AxisConfig(
    labelFontSize = 16,
    titleFontSize = 18,
    titleColor = "#364547"),
  legend = alt.LegendConfig(
    titleFontSize = 18,
    labelFontSize = 16,
    titleColor = "#364547"),
  font = 'Roboto')

plot

alt.Chart(...)

# 2. [50 puntos] Utilizando la biblioteca `altair` y conjunto de datos base de datos `IPS_cantonal.xlsx` realice lo siguiente:

In [2]:
datos = pd.read_excel('../datos/base-de-datos-IPS-cantonal.xlsx')
datos

/Users/promidat04/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Cve,Cantón,Nutrición y Asistencia Médica Básica,Agua y Saneamiento,Vivienda,Seguridad Personal,Acceso a Conocimientos Básicos,Acceso a Información y Comunicaciones,Salud y Bienestar,Sustentabilidad del Ecosistema,...,Rendición de cuentas,Acceso a la cultura y recreación,Inclusión al adulto mayor,Inclusión a población con discapacidad,Inclusión población indígena,Inclusión de migrantes,Inclusión de hogares con jefatura femenina,Población con educación superior,Escolaridad promedio de las mujeres,Población con educación técnica
0,101,SAN JOSE,82.987394,98.665964,63.891581,43.713222,84.284809,73.758822,64.570213,39.810430,...,95.3,20.552,5.252235,7.709731,7.919016,25.562775,36.874519,29.469448,9.733187,4.329589
1,102,ESCAZU,85.931253,98.121264,86.023111,52.939193,88.650664,77.359311,67.819220,64.554111,...,90.0,15.573,6.169883,6.741573,6.495726,10.226506,29.850746,40.279616,10.583268,4.197981
2,103,DESAMPARADOS,88.541844,94.700064,80.789600,58.999818,88.804140,70.986816,68.089101,48.467391,...,33.3,3.071,5.102137,8.366314,9.358289,18.081342,31.065089,21.696907,9.038890,5.288635
3,104,PURISCAL,90.993459,89.766092,90.227325,86.085942,84.829898,58.456934,76.552294,65.577701,...,97.7,5.454,8.805477,6.519700,8.341232,27.045770,17.241379,20.662035,8.512712,7.423228
4,105,TARRAZU,91.495155,96.414943,88.021404,65.438845,77.763379,55.222121,77.395928,71.499005,...,81.0,8.600,4.359313,10.572391,33.012821,25.316456,11.111111,12.510132,7.431191,3.440599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,702,POCOCI,93.710271,88.383142,74.219748,52.284697,85.629427,56.824826,81.059653,53.672129,...,82.0,3.425,13.157369,12.746757,13.468939,25.725650,24.083770,11.816294,7.428762,4.176034
77,703,SIQUIRRES,95.130944,62.759771,78.607393,51.885455,84.434846,54.077897,77.828522,57.988991,...,97.3,1.937,13.290150,14.055829,17.743702,20.353282,15.277778,9.480705,7.287186,5.604915
78,704,TALAMANCA,84.478978,80.812963,26.828208,58.506535,68.188011,42.608662,69.068280,76.325130,...,66.7,6.187,29.541864,17.794572,30.706153,28.525512,30.000000,10.315186,6.904943,4.836683
79,705,MATINA,85.092291,90.308685,64.382709,49.398883,80.095213,49.114978,83.942474,61.683156,...,66.7,0.265,18.428751,14.405915,28.922495,23.230386,15.151515,7.153248,6.892898,4.733763


In [3]:
datos["Tasa de robos y asaltos"]

0     717.666962
1     174.150428
2     189.400740
3       8.699435
4      64.117510
         ...    
76    155.471215
77    205.379621
78     44.600546
79    191.989890
80     83.910667
Name: Tasa de robos y asaltos, Length: 81, dtype: float64

## a) Diseñe un gráfico que permita analizar la relación entre Tasa de robos y asaltos y Rezago escolar para los distintos cantones.

In [4]:
def quantile_norm(df_input):
    sorted_df = pd.DataFrame(np.sort(df_input.values,axis=0), index=df_input.index, columns=df_input.columns)
    mean_df = sorted_df.mean(axis=1)
    mean_df.index = np.arange(1, len(mean_df) + 1)
    quantile_df =df_input.rank(method="min").stack().astype(int).map(mean_df).unstack()
    return(quantile_df)

## b) Utilizando la biblioteca `altair` replique el diseño del punto anterior.

In [5]:
alt.Chart(datos,title = "Relación Tasa de robos y asaltos con Rezago escolar."
                 ).mark_point().encode(
    x = alt.X('Tasa de robos y asaltos:Q', title = "Robos y Asaltos"),
    y = alt.Y('Rezago escolar:Q', title = "Rezago escolar"),
    color = alt.Color('Cantón', 
                      legend = alt.Legend(title = "Cantón")))

alt.Chart(...)

## c) Interprete los resultados.

## d) Diseñe un gráfico que permita analizar la relación entre Acceso a la cultura y recreación y Acceso a Educación Superior para los distintos cantones agrupando en 5 grupos según su Índice de Progreso Social, para crear los grupos puede utilizar las técnicas vista en la clase 2.

## e) Utilizando la biblioteca `altair` replique el diseño del punto anterior.

In [6]:
datos['grupos'] = pd.cut(x = datos['Índice de Progreso Social']  , bins = 5)
datos['grupos'] = datos["grupos"].astype('str')

In [7]:
alt.Chart(datos,title = "Relación de Acceso a la cultura y Acceso a Educación Superior."
                 ).mark_point().encode(
    x = alt.X('Acceso a la cultura y recreación:Q', title = "Acceso a la cultura y recreación"),
    y = alt.Y('Acceso a Educación Superior:Q', title = "Acceso a Educación Superior"),
    color = alt.Color('grupos', 
                      legend = alt.Legend(title = "Grupos")))

alt.Chart(...)

## f) Interprete los resultados.

# 3. [20 puntos] Utilizando la biblioteca `altair` y los datos de su preferencia diseñe al menos 3 gráficos siguiendo los siguiente requerimientos:

## a) Uso de una paleta de colores personalizada.

## b) Uso de anotaciones como título, subtítulo, etc.

## c) Uso de selectores y tranformadores

In [20]:
data.list_datasets()

['7zip',
 'airports',
 'annual-precip',
 'anscombe',
 'barley',
 'birdstrikes',
 'budget',
 'budgets',
 'burtin',
 'cars',
 'climate',
 'co2-concentration',
 'countries',
 'crimea',
 'disasters',
 'driving',
 'earthquakes',
 'ffox',
 'flare',
 'flare-dependencies',
 'flights-10k',
 'flights-200k',
 'flights-20k',
 'flights-2k',
 'flights-3m',
 'flights-5k',
 'flights-airport',
 'gapminder',
 'gapminder-health-income',
 'gimp',
 'github',
 'graticule',
 'income',
 'iowa-electricity',
 'iris',
 'jobs',
 'la-riots',
 'londonBoroughs',
 'londonCentroids',
 'londonTubeLines',
 'lookup_groups',
 'lookup_people',
 'miserables',
 'monarchs',
 'movies',
 'normal-2d',
 'obesity',
 'ohlc',
 'points',
 'population',
 'population_engineers_hurricanes',
 'seattle-temps',
 'seattle-weather',
 'sf-temps',
 'sp500',
 'stocks',
 'udistrict',
 'unemployment',
 'unemployment-across-industries',
 'uniform-2d',
 'us-10m',
 'us-employment',
 'us-state-capitals',
 'volcano',
 'weather',
 'weball26',
 'wheat',

In [115]:
countries = data.countries()
countries = countries.iloc[:,1:9] #me deshago de la primera col
countries = countries.dropna()
countries.year = pd.to_datetime(countries.year,format = '%Y')
countries

,year,fertility,life_expect,n_fertility,n_life_expect,country,p_fertility,p_life_expect
1,1960-01-01,7.7000,31.997,7.7000,34.020,Afghanistan,7.7000,30.332
2,1965-01-01,7.7000,34.020,7.7000,36.088,Afghanistan,7.7000,31.997
3,1970-01-01,7.7000,36.088,7.7000,38.438,Afghanistan,7.7000,34.020
4,1975-01-01,7.7000,38.438,7.8000,39.854,Afghanistan,7.7000,36.088
5,1980-01-01,7.8000,39.854,7.9000,40.822,Afghanistan,7.7000,38.438
...,...,...,...,...,...,...,...,...
624,1975-01-01,4.4685,67.456,3.9570,68.557,Venezuela,4.9410,65.712
625,1980-01-01,3.9570,68.557,3.6485,70.190,Venezuela,4.4685,67.456
626,1985-01-01,3.6485,70.190,3.2500,71.150,Venezuela,3.9570,68.557
627,1990-01-01,3.2500,71.150,2.9415,72.146,Venezuela,3.6485,70.190


In [170]:
countries['year'].unique

<bound method Series.unique of 1     1960-01-01
2     1965-01-01
3     1970-01-01
4     1975-01-01
5     1980-01-01
         ...    
624   1975-01-01
625   1980-01-01
626   1985-01-01
627   1990-01-01
628   1995-01-01
Name: year, Length: 504, dtype: datetime64[ns]>

In [194]:
year_slider = alt.binding_range(min=1960, max=1995, step=5)
slider_selection = alt.selection_single(bind = year_slider,
                                        fields = ['year'],
                                        name = "Select", init={'year': 50})

In [195]:



alt.Chart(countries,
                 title = "Relación año y expectativa de vida de países."
                 ).mark_circle(point = { "size": 80},
                             strokeWidth = 2,
                             color = "#c67ace").encode(
    x = alt.X('year(year):T', title = "Año"),
    y = alt.Y('mean(n_life_expect):Q', 
              title = "Expectativa de Vida",
              scale = alt.Scale(zero = False)),
    color = alt.Color('country', 
                      legend = alt.Legend(title = "País")),
    tooltip = [slider_selection.year,
               'min(n_life_expect):Q',
               'mean(n_life_expect):Q',
               'median(n_life_expect):Q',
               'max(n_life_expect):Q']).properties(width=650, height=400
    ).add_selection(
    slider_selection).transform_filter(
    slider_selection).transform_calculate(year = "year(datum.year)")
                      


SchemaValidationError: Invalid specification

        altair.vegalite.v4.schema.core.FacetedEncoding->2->items, validating 'type'

        'Select.year' is not of type 'object'
        

alt.Chart(...)

In [ ]:
plot = plot.configure(
  title = alt.TitleConfig(
    fontSize = 22,
    align = "center",
    color = "#364547"),
  axis = alt.AxisConfig(
    labelFontSize = 16,
    titleFontSize = 18,
    titleColor = "#364547"),
  legend = alt.LegendConfig(
    titleFontSize = 18,
    labelFontSize = 16,
    titleColor = "#364547"),
  font = 'Roboto')

plot